# Harbor / GoHarbor Case Study

This is a Reagent Analytics project demo, using the API, to answer questions about foreign influence in various entities in the Harbor open source project.

The `reagentpy` Python package facilitates use of the [Reagent API](https://api.reagentanalytics.com) and querying the Reagent version control intelligence database.

## Import the `Reagent` object

In [1]:
from reagentpy import Reagent

Check the status of the connection.

In [2]:
Reagent().status().dict()

{'status': 'ok'}

Set all required variable names to appropriately constrain query results.

In [3]:
repo_name = "goharbor/harbor"
limit = 50
china_tz = 8.0

# Non-Adversarial Threat

There are a number of human factors that influence the likelihood that vulnerabilities are injected into repositories at various points in the open source lifecycle.  These can be used to infer threat that can be quantified, and thus analyzed.  These metrics are all on a sliding scale between from **0 - 10**, 0 meaning that Harbor is unaffected by this issue, and 10 meaning that that all commits exhibit the problem at hand.

## Threatscoring Breakdown

**Project Fragmentation** is defined as the ratio of files within a repository edited by more than ten developers to files within a repository edited by less than ten developers.  The significance factor for this function is **r = 16**.

**Unfocused Contribution** is measured by taking the average pagerank of each file within a repo.  The significance factor for this function is **r = .4497**.

**Context Switching** is calculated by taking the average weekly density of distinct file communities users commit to each week.  The significance factor for this function is **r = 0.17**.

**Interactive Churn** is the average weekly number of user interactions a file has, scaled by how recent each action is.  The significance factor for this function is **r = 0.16**.

In [ ]:
get_threatscores_rows_response = Reagent().enrichments().threat_score(repo_name)
get_threatscores_rows_response.df()
Reagent().demo_visualizations().create_out_of_five_chart(repo_name)

## Foreign Influence

The goal of this project is to find foreign adversarial influence in the codebase, wherever it may exist.  To start, let's look at a very high-level analysis of what that means in this project.

In [ ]:
Reagent().enrichments().foreign_influence(repo_name).df()

## What organizations are involved?  What are the affiliations of those organizations?

Analyzing email domains found in the repo yeilds a glimpse into the organizations who are interested in using Harbor.

In [ ]:
response = Reagent().repo().email_domains(repo_name)
response.df()
Reagent().demo_visualizations().wordcloud(response)

## Repository Health

Here are some indications of whether or not an open source repository is well-maintained, and is following the most basic security guidelines set by GitHub.

This endpoint is not currently available, but we anticipate that it will be ready shortly.

In [4]:
response = Reagent().repo().hygiene_summary(repo_name)
response.df()
Reagent().demo_visualizations().print_hygiene_summary(response)

AttributeError: 'RepoClient' object has no attribute 'hygiene_summary'

In [5]:
response = Reagent().repo().repo_list(limit)
response.df()

AttributeError: 'RepoClient' object has no attribute 'repo_list'

# Adversarial Risk

Describes deliberate attacks by bad actors with intent, capability, and targeting characteristics of any scale.  While all kinds of risk are invaluable in deciding how "safe" a resource is to use, adversarial risk serves as the best way to quantify sources of attack.

### Timezones

Often, one can deduce national origin by simply looking at timezones.  This is helpful when analyzing entities that aren't large or old enough to have country data associated with it.  Here's the breakdown for this repo according to timezones.

In [ ]:
response = Reagent().repo().timezones(repo_name)
response.df()
response["timezone_commit_totals"].df()

In [ ]:
Reagent().timezone_visualizations().show_logarithmic_bar_chart(response)

## Identity Verification

One prime indicator of adversarial intent is the deliberate concealment of identity.  We have several methods to detect this in users.

### Timezone Manipulation

Manipulating where you seem to be from is an indication of open source malpractice - specifically, manipulating your location incorrectly.  We can find instances of "teleportation" in our database by finding commits from different timezones impossibly close together in time.

In [ ]:
Reagent().enrichments().timezone_spoof(repo_name).df()

## Data Breaches

Almost no one in Harbor has been a part of a database breach - all email accounts appear to be purpose-built work accounts.  Here's what that looks like:

In [ ]:
response = Reagent().enrichments().hibp(repo_name)
response.df()
Reagent().demo_visualizations().hibp_pie_chart(response)